# TM10007 Assignment template

In [6]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [7]:
import numpy as np
from sklearn.impute import KNNImputer

from brats.load_data import load_data

In [8]:
# Load data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


The number of samples: 167
The number of columns: 725


In [9]:
# Count number of NaN values for each row and column
no_nan_row = data.isnull().sum(axis=1)
no_nan_col = data.isnull().sum(axis=0)

# Drop column containing patient labels for imputation
data_imp = data.drop(columns=['label'])

# Impute NaN values using k-NearestNeighbor
imputer = KNNImputer(n_neighbors=5, weights="uniform")
array_imp = imputer.fit_transform(data_imp)

data_imp[:] = array_imp
data_imp['label'] = data['label']
#no_nan_array = np.count_nonzero(np.isnan(array_imp))
#no_nan_row_imp = data_imp.isnull().sum().sum()





In [10]:
# Determining the number of NaN for GBM and LGG seperately
# It can be usefull to see from which the class the NaN's come from
# This should/might be taken into account when selecting the features

import pandas as pd

GBM = data.loc[data['label'] == 'GBM']
GBM_no_nan_col = GBM.isnull().sum(axis=0)
LGG = data.loc[data['label'] == 'LGG']
LGG_no_nan_col = LGG.isnull().sum(axis=0)

frame = { 'Total (n=167)': no_nan_col, 'GBM (n=102)': GBM_no_nan_col, 'LGG (n=65)': LGG_no_nan_col } 
overview_nan = pd.DataFrame(frame)